En mi proyecto trabajare con el registro publico de defunciones año a año, los datos fueron extraidos de la pagina de datos abiertos de mexico: https://datos.gob.mx/busca/dataset/defunciones

In [ ]:
# Transformacion de archivos individuales a archivo comunal con todas las defunciones

In [2]:
import pandas as pd 
import os 
import glob 

In [3]:
path=r"C:\Users\Victor\OneDrive\Documentos\Datos_masivos\Defunciones"

In [4]:

csv_files = glob.glob(os.path.join(path, "*.csv")) 
  
lista=[]

for f in csv_files: 
      
    # read the csv file 
    df = pd.read_csv(f)
    df.columns = map(str.lower, df.columns)
    lista.append(df)

C:\Users\Victor\AppData\Local\Temp\ipykernel_48536\2167083577.py:8: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
C:\Users\Victor\AppData\Local\Temp\ipykernel_48536\2167083577.py:8: DtypeWarning: Columns (42,50) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)
C:\Users\Victor\AppData\Local\Temp\ipykernel_48536\2167083577.py:8: DtypeWarning: Columns (72) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f)


In [5]:
datos_no_transformados=pd.concat(lista, axis=0, ignore_index=True)

In [6]:
datos_no_transformados.to_csv("Datos_concatenados_no_convertidos.csv")

In [8]:
# Importacion a pyspark

In [7]:
import pyspark

import findspark
findspark.init()
findspark.find()

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

In [9]:
spark= SparkSession.builder.appName("Datos_defunciones").getOrCreate()

In [10]:
df=spark.read.csv("Datos_concatenados_no_convertidos.csv", header=True, inferSchema=True)

In [11]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- ent_regis: integer (nullable = true)
 |-- mun_regis: integer (nullable = true)
 |-- ent_resid: integer (nullable = true)
 |-- mun_resid: integer (nullable = true)
 |-- tloc_resid: integer (nullable = true)
 |-- ent_ocurr: integer (nullable = true)
 |-- mun_ocurr: integer (nullable = true)
 |-- causa_def: string (nullable = true)
 |-- lista_mex: string (nullable = true)
 |-- sexo: integer (nullable = true)
 |-- edad: integer (nullable = true)
 |-- dia_ocurr: integer (nullable = true)
 |-- mes_ocurr: integer (nullable = true)
 |-- anio_ocur: integer (nullable = true)
 |-- mes_regis: integer (nullable = true)
 |-- anio_regis: integer (nullable = true)
 |-- dia_nacim: integer (nullable = true)
 |-- mes_nacim: integer (nullable = true)
 |-- anio_nacim: integer (nullable = true)
 |-- ocupacion: integer (nullable = true)
 |-- escolarida: integer (nullable = true)
 |-- edo_civil: integer (nullable = true)
 |-- presunto: double (nullable = true)
 |-

In [12]:
df.show()

+---+---------+---------+---------+---------+----------+---------+---------+---------+---------+----+----+---------+---------+---------+---------+----------+---------+---------+----------+---------+----------+---------+--------+----------+----------+---------+----------+----------+---------+----------+----------+----------+--------+--------+-----+-------+--------+-----+------+---------+--------+---------+--------+----------+----------+---------+-------+----------+--------+--------+---------+----+------+--------+--------+----------+----------+---------+---------+----------+-------+--------+----------+---------+----------+-------+-------+--------+----------+----------+--------+------+----------+-------+--------+----------+----------+-------+--------+
|_c0|ent_regis|mun_regis|ent_resid|mun_resid|tloc_resid|ent_ocurr|mun_ocurr|causa_def|lista_mex|sexo|edad|dia_ocurr|mes_ocurr|anio_ocur|mes_regis|anio_regis|dia_nacim|mes_nacim|anio_nacim|ocupacion|escolarida|edo_civil|presunto|ocurr_trab|lu

In [13]:
df.describe().show()

+-------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+---------+------------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+-----------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+-----------------+-----------------+------------------+------------------+------------------+--------+-----------------+------------------+-----------------+------------------+------------------+------------------+-----------------+------------------+-----------------+-----------------+

In [16]:
#Obteniendo las defunciones unicamente de nuevo leon (ent_regis=19)
df.filter(df.ent_regis ==19).orderBy(F.desc("anio_regis")).show()

+--------+---------+---------+---------+---------+----------+---------+---------+---------+---------+----+----+---------+---------+---------+---------+----------+---------+---------+----------+---------+----------+---------+--------+----------+----------+---------+----------+----------+---------+----------+----------+----------+--------+--------+-----+-------+--------+-----+------+---------+--------+---------+--------+----------+----------+---------+-------+----------+--------+--------+---------+----+------+--------+--------+----------+----------+---------+---------+----------+-------+--------+----------+---------+----------+-------+-------+--------+----------+----------+--------+------+----------+-------+--------+----------+----------+-------+-----------+
|     _c0|ent_regis|mun_regis|ent_resid|mun_resid|tloc_resid|ent_ocurr|mun_ocurr|causa_def|lista_mex|sexo|edad|dia_ocurr|mes_ocurr|anio_ocur|mes_regis|anio_regis|dia_nacim|mes_nacim|anio_nacim|ocupacion|escolarida|edo_civil|presunto|

In [23]:
#Viendo evolucion año a año
df.filter(df.ent_regis == 19).groupBy("anio_regis").count().show()

+----------+-----+
|anio_regis|count|
+----------+-----+
|      2001|17231|
|      2000|17031|
|      2002|17642|
|      2003|18086|
|      2004|18078|
|      2005|19928|
|      2006|19371|
|      2007|20742|
|      2008|21109|
|      2009|23167|
|      2010|23862|
|      2011|24616|
|      2012|24651|
|      2013|25127|
|      2014|25407|
|      2015|25723|
|      2016|26783|
|      2017|27919|
|      2018|29686|
|      2019|31065|
+----------+-----+
only showing top 20 rows

